In [1]:
import numpy as np
from pyscf import gto, scf, dft

from pyscf.tools.dump_mat import dump_rec, dump_mo
from sys import stdout

from pop_scheme import FrgPopulationScheme, FrgMullikenPopulation
from rdft       import Constraints, rdft

mol = gto.Mole()
mol.verbose = 0
mol.atom = '''
O  -1.551007  -0.114520   0.000000
H  -1.934259   0.762503   0.000000
H  -0.599677   0.040712   0.000000
O   1.350625   0.111469   0.000000
H   1.680398  -0.373741  -0.758561
H   1.680398  -0.373741   0.758561
'''

mol.basis = '6-31g(d)'
mol.spin   = 0
mol.charge = 0
mol.cart   = True
mol.build()

In [9]:
omega        = 40.0
frg_list            = [[0,1,2],[3,4,5]]
nelec_required_list = [[10.0], [10.0]]

mf           = scf.RHF(mol)
mf.conv_tol  = 1e-9
mf.verbose   = 0
mf.max_cycle = 100
mf.kernel()

dump_mo(mol, mf.mo_coeff)
dump_rec(stdout, mf.mo_coeff)

etot_0             = mf.e_tot
old_get_fock       = mf.get_fock
old_energy_elec    = mf.energy_elec
dm = mf.make_rdm1()

omega_vals          = [[omega],[0.0]]
e_add1              = rdft(mf, frg_list, nelec_required_list, omega_vals, dm0=dm, verbose=0, maxiter=500, tol=1e-8,
                               old_energy_elec=old_energy_elec, old_get_fock=old_get_fock)
    
pop                 = mf.mulliken_pop(verbose=4)[1]
pop1                = pop[0]+pop[1]+pop[2]
etot_1              = mf.e_tot

print("omega = {: 12.6f}, etot_1 = {: 12.6f}, etot_0 = {: 12.6f}, e_add1 = {: 12.6f}, pop1 = {: 12.6f}".format(
        omega, etot_1, etot_0, e_add1, pop1
    ))

               #0        #1        #2        #3        #4       
 0  O 1s      -0.00008   0.99463  -0.01392  -0.20988  -0.00000
 0  O 2s      -0.00014   0.02114   0.03369   0.47130   0.00000
 0  O 3s       0.00069   0.00418   0.01908   0.45408   0.00000
 0  O 2px     -0.00003   0.00056   0.00637   0.04229   0.00000
 0  O 2py      0.00000   0.00115   0.00523   0.08108  -0.00000
 0  O 2pz     -0.00000   0.00000   0.00000   0.00000   0.00423
 0  O 3px      0.00027  -0.00016  -0.00085   0.02877   0.00000
 0  O 3py      0.00003  -0.00040   0.00238   0.04341  -0.00000
 0  O 3pz      0.00000  -0.00000  -0.00000   0.00000   0.00454
 0  O 3dxx    -0.00002  -0.00263   0.00164   0.01516   0.00000
 0  O 3dxy     0.00001   0.00007  -0.00014  -0.00274   0.00000
 0  O 3dxz    -0.00000  -0.00000  -0.00000   0.00000   0.00077
 0  O 3dyy    -0.00006  -0.00260   0.00149   0.01310  -0.00000
 0  O 3dyz    -0.00000   0.00000  -0.00000   0.00000   0.00028
 0  O 3dzz    -0.00007  -0.00249   0.00058  -0.00172 

In [8]:
dump_mo(mol, mf.mo_coeff)

               #0        #1        #2        #3        #4       
 0  O 1s      -0.00007   0.99463  -0.00818  -0.21028  -0.00000
 0  O 2s      -0.00019   0.02114   0.01525   0.47174  -0.00000
 0  O 3s       0.00097   0.00418   0.02385   0.45630   0.00000
 0  O 2px     -0.00005   0.00056   0.00370   0.04246  -0.00000
 0  O 2py      0.00000   0.00115   0.00266   0.08121   0.00000
 0  O 2pz     -0.00000   0.00000  -0.00000  -0.00000   0.00339
 0  O 3px      0.00038  -0.00016   0.00618   0.02967  -0.00000
 0  O 3py      0.00004  -0.00040   0.00158   0.04358   0.00000
 0  O 3pz     -0.00000  -0.00000   0.00000  -0.00000   0.00361
 0  O 3dxx    -0.00005  -0.00263   0.00002   0.01511  -0.00000
 0  O 3dxy     0.00001   0.00007   0.00006  -0.00271  -0.00000
 0  O 3dxz     0.00000   0.00000  -0.00000  -0.00000   0.00072
 0  O 3dyy    -0.00009  -0.00260  -0.00050   0.01298  -0.00000
 0  O 3dyz    -0.00000  -0.00000   0.00000  -0.00000   0.00023
 0  O 3dzz    -0.00010  -0.00248  -0.00103  -0.00187 